## 开始

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/My\ Drive/Colab Notebooks/OursRepository/public-opinion-monitor
# 
# !pip install thulac
# !pip install jieba

In [5]:
from ClassicalHANModel import *
import torch
import torch.nn as nn
from torch.optim.optimizer import Optimizer
import numpy
import time, math
import torch.utils.data as data
import os
import pandas as pd
import json
import thulac
thulac = thulac.thulac()
import jieba

Model loaded succeed


In [6]:
# import torch.functional as F
# 
embedding = nn.Embedding(10, 3)
input = torch.LongTensor([[0, 1,2,4,5],[1, 4,3,2,9]])
print(embedding(input))
embedding2 = nn.Embedding(10, 3, padding_idx=2)
# input = torch.LongTensor([[1,2,0,5, 6,7,8,9]])
print(embedding2(input))
weight = torch.FloatTensor([[1, 2.3, 3], [4, 5.1, 6.3]])
embedding3 = nn.Embedding.from_pretrained(weight)
input = torch.LongTensor([1])
print(embedding3(input))
input = torch.LongTensor([0])
print(embedding3(input))



tensor([[[-0.4567, -0.2409, -0.8837],
         [-0.5701, -0.9491, -2.7943],
         [ 1.5182, -0.5346, -0.6047],
         [-0.0546, -0.0513, -1.0322],
         [-0.5860,  0.0557, -1.3539]],

        [[-0.5701, -0.9491, -2.7943],
         [-0.0546, -0.0513, -1.0322],
         [-1.1007, -1.0440,  0.1437],
         [ 1.5182, -0.5346, -0.6047],
         [ 1.7443,  0.5313, -0.0597]]], grad_fn=<EmbeddingBackward>)
tensor([[[-0.7053,  1.0738, -0.8784],
         [-0.5785,  1.6240, -0.4461],
         [ 0.0000,  0.0000,  0.0000],
         [-0.4914,  0.4058, -2.5656],
         [-0.4140,  0.6332,  0.9174]],

        [[-0.5785,  1.6240, -0.4461],
         [-0.4914,  0.4058, -2.5656],
         [-2.1385,  1.1311, -0.5590],
         [ 0.0000,  0.0000,  0.0000],
         [-0.5943,  0.0329,  0.3941]]], grad_fn=<EmbeddingBackward>)
tensor([[4.0000, 5.1000, 6.3000]])
tensor([[1.0000, 2.3000, 3.0000]])


## 读取词向量
建立词语列表

In [7]:
from gensim.models import KeyedVectors, Word2Vec

# file = '../../PretrainedData/Tencent_AILab_ChineseEmbedding/Tencent_AILab_ChineseEmbedding.txt'
# file = '../../DataSets/Word2Vect/xingrong_50_thulac/word2vect_50_w5.model'
# file = '../../DataSets/Word2Vect/xiejunjie_300_jieba/wiki_han_word2vec_300维度.model'
file = '../../DataSets/Word2Vect/Tencent_AILab_ChineseEmbedding/Tencent_AILab_ChineseEmbedding_Min.txt'
# word2vec = Word2Vec.load(file)
word2vec = KeyedVectors.load_word2vec_format(file, binary=False,limit=100000)
# word2vec = KeyedVectors.load_word2vec_format(file, binary=False)
word2vec.init_sims(replace=True)  # 神奇，很省内存，可以运算most_similar
word2vec.vector_size

unable to import 'smart_open.gcs', disabling that module


200

In [8]:
print(word2vec)
# print(word2vec.wv.vocab)
# print(len(word2vec.index2word))
print(len(word2vec.wv.index2word))
print(word2vec.wv.index2word[0])
print(word2vec.wv.index2word[1])
print(word2vec.wv.index2word[2])
print(word2vec.wv.index2word[1522])
print(word2vec.wv.index2entity[1522])
print(word2vec.similar_by_word('中国'))
print(word2vec.similar_by_word('天才'))
print(word2vec.wv)
print('word2vec.wv.vocab ---- >', word2vec.wv.vocab)
print(word2vec.wv.index2word)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

C:\Users\fuliu\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.
C:\Users\fuliu\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """
C:\Users\fuliu\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
C:\Users\fuliu\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:7: Deprecation

In [16]:
wordEmbedding = [word2vec.wv[word]  for word in word2vec.wv.index2word]
word2index = { word:i for i, word in enumerate(word2vec.wv.index2word)}
# print(wordEmbedding[:2])
# print(word2index['中国'])
# print(word2index['天才'])
print(word2index['喜欢'])
print(word2index['不'])
print(word2index['爽'])
print(word2index['超赞'])



C:\Users\fuliu\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.
C:\Users\fuliu\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


217
64
5313
41047


## 读取训练数据

In [9]:
def compute_ngrams(word, num_min = 1, num_max = 3):
    ngrams =[]
    for ngram_length in range(num_min, min(len(word), num_max) + 1):
        for i in range(len(word) - ngram_length + 1):
            # print(i, i + ngram_length)
            ngrams.append(word[i : i + ngram_length])
    # print(ngrams)
    return list(set(ngrams))

print(compute_ngrams('you'))
print(compute_ngrams('I Think'))
print(compute_ngrams('中华人民共和国万岁'))

['u', 'yo', 'y', 'o', 'ou', 'you']
['T', 'n', 'i', 'in', ' T', 'hin', 'Thi', ' ', 'k', 'I ', 'Th', 'hi', 'nk', 'ink', 'h', 'I T', ' Th', 'I']
['国万', '共和国', '和国', '岁', '民共', '国', '人民共', '华人', '人', '共和', '中', '中华', '华人民', '中华人', '人民', '华', '共', '国万岁', '和国万', '民共和', '民', '万', '万岁', '和']


In [10]:
# 从词向量文本文件 word2vec 中获取词向量，如果获取到直接返回，若没有获取到，那么把这个词拆开
# 成为 ngrams 的新词组，并在 word2vec 中找新词组中的词向量并相加取平均，最后得到平均词向量输出
def wordVec(word, word2vec, min_n = 1, max_n = 3):
    # 确认词向量维度
    word_size = word2vec.wv.syn0[0].shape[0]

    # 如果在词典之中，直接返回词向量
    if word in word2vec.wv.vocab.keys():
        return word2vec[word]
    else:
        # 计算word的ngrams词组
        ngrams = compute_ngrams(word, min_n, max_n)
        # 不在词典的情况下
        word_vec = numpy.zeros(word_size, dtype=numpy.float32)
        ngrams_found = 0
        ngrams_single = [ng for ng in ngrams if len(ng) == 1]
        ngrams_more = [ng for ng in ngrams if len(ng) > 1]
        # 先只接受2个单词长度以上的词向量
        for ngram in ngrams_more:
            if ngram in word2vec.wv.vocab.keys():
                word_vec += word2vec[ngram]
                ngrams_found += 1
                #print(ngram)
        # 如果，没有匹配到，那么最后是考虑单个词向量
        if ngrams_found == 0:
            for ngram in ngrams_single:
                word_vec += word2vec[ngram]
                ngrams_found += 1
        if word_vec.any():
            return word_vec / max(1, ngrams_found)
        else:
            # 不抛出异常，而是打印提示，并返回0向量。
            print(KeyError('all ngrams for word %s absent from model' % word))
            return word_vec

In [11]:
class DotDict(dict):
    def __init__(self, *args, **kwargs):
        dict.__init__(self, *args, **kwargs)
        self.__dict__ = self
        

In [12]:
import pandas as pd

dic = {'a':[1, 2, 3, 4], 'b':[5, 6, 7, 8],
'c':[9, 10, 11, 12], 'd':[13, 14, 15, 16]}
df1=pd.DataFrame(dic)
print(df1)
df2=df1.sample(frac=0.75)
print(df2)
# rowlist=[]
# for indexs in df2.index:
#     rowlist.append(indexs)
df3=df1.drop(df2.index.to_list(),axis=0)
print(df3)

   a  b   c   d
0  1  5   9  13
1  2  6  10  14
2  3  7  11  15
3  4  8  12  16
   a  b   c   d
3  4  8  12  16
1  2  6  10  14
2  3  7  11  15
   a  b  c   d
0  1  5  9  13


In [13]:
import re

def splitText(text:str, splitChar = '(。|，|,|！|\!|？|\?|\n|\t)'):
  '''
  句子切分分隔符
  '''
  contents = re.split(splitChar, text)
  contents = ["".join([a, b]) if b != '\n' and b != '\t' else "" #a + "。"
              for a, b in zip(contents[0::2], contents[1::2])]
  contents = [content for content in contents if content.strip() != '' and content[0] != '。']
  # for i, sen in enumerate(contents):
  #     print(i, sen)
  return contents

testStr = """
深圳，简称“深”，别称鹏城，是广东省副省级市、计划单列市、超大城市，国务院批复确定的中国经济特区、全国性经济中心城市和国际化城市 [1]  。截至2018年末，全市下辖9个区，总面积1997.47平方千米，建成区面积927.96平方千米，常住人口1302.66万人，城镇人口1302.66万人，城镇化率100%，是中国第一个全部城镇化的城市。 [2-5] 
深圳地处中国华南地区、广东南部、珠江口东岸，东临大亚湾和大鹏湾，西濒珠江口和伶仃洋，南隔深圳河与香港相连，是粤港澳大湾区四大中心城市之一 [6]  、国家物流枢纽、国际性综合交通枢纽 [7]  、国际科技产业创新中心 [8]  、中国三大全国性金融中心之一 [9-10]  ，并全力建设中国特色社会主义先行示范区 [11]  、综合性国家科学中心 [12]  、全球海洋中心城市 [13]  。深圳水陆空铁口岸俱全，是中国拥有口岸数量最多、出入境人员最多、车流量最大的口岸城市。 [14]
"""

print(splitText(testStr))

['深圳，', '简称“深”，', '别称鹏城，', '是广东省副省级市、计划单列市、超大城市，', '国务院批复确定的中国经济特区、全国性经济中心城市和国际化城市 [1]  。', '截至2018年末，', '全市下辖9个区，', '总面积1997.47平方千米，', '建成区面积927.96平方千米，', '常住人口1302.66万人，', '城镇人口1302.66万人，', '城镇化率100%，', '是中国第一个全部城镇化的城市。', '深圳地处中国华南地区、广东南部、珠江口东岸，', '东临大亚湾和大鹏湾，', '西濒珠江口和伶仃洋，', '南隔深圳河与香港相连，', '是粤港澳大湾区四大中心城市之一 [6]  、国家物流枢纽、国际性综合交通枢纽 [7]  、国际科技产业创新中心 [8]  、中国三大全国性金融中心之一 [9-10]  ，', '并全力建设中国特色社会主义先行示范区 [11]  、综合性国家科学中心 [12]  、全球海洋中心城市 [13]  。', '深圳水陆空铁口岸俱全，', '是中国拥有口岸数量最多、出入境人员最多、车流量最大的口岸城市。']


In [14]:
def isNan(a):
    return a != a


class RatingData(data.Dataset):
    def __init__(self, path, 
          word2index, 
          max_row = -1, 
          num_word_in_sentence = 10,
          num_sentence_in_article = 10,
          trainTestRate = 0.85, 
          isTrain = True, 
          wordCuter = thulac,
          clean_file_name = 'ratings_clean_4_HAN.csv',
          ):
        self.token_list = []
        self.label_list = []
        # self.token_positions = torch.tensor([i for i in range(100)])

        print(' balance_data.csv 所在path:',path) # 地址不应该包含 ratings.csv

        ratings_clean_filename = os.path.join(path, clean_file_name)
        ratings_filename = os.path.join(path, 'balance_data_means_600000.csv')
        if os.path.isfile(ratings_clean_filename):
            clean_pd = pd.read_csv(ratings_clean_filename)
        else:
            print('没有找到缓存的文件%s, 读取源文件%s'%(ratings_clean_filename, ratings_filename))
            ratings_pd = pd.read_csv(ratings_filename)
            print('开始生成缓存文件%s'%(ratings_clean_filename))
            clean_pd = pd.DataFrame({
                'labels':[],
                'data':[],
            })
            nonRatingCount = 0
            for i, row in ratings_pd.iterrows():
                if max_row != -1 and i > max_row:
                    break
                if not isinstance(row['data'], str) or row['data'] == '':
                    # print(i + 1, row['comment'])
                    nonRatingCount += 1
                    continue
                r0 = row['labels']
                if r0 == -1:
                    r0 = 2
                    
                if i % 10000 == 9999:
                    print(i + 1, r0)

                # 把文章切割成句子
                sentences = splitText(row['data'])
                tokens = []
                for i in range(num_sentence_in_article):
                  if i < len(sentences):
                    # 把句子切割成词
                    words = list(wordCuter.cut(sentences[i]))
                    # 把词转成词向量中的index，不足时补全
                    token = [ word2index[words[j]] if j < len(words) and words[j] in word2index else 0 
                          for j in range(num_word_in_sentence)] 
                    tokens += token
                  else:
                    # 不足时补全
                    tokens += [0] * num_word_in_sentence
                
                newRow = DotDict()

                newRow.labels = [r0]
                newRow.data = [json.dumps(tokens)]

                clean_pd = clean_pd.append(pd.DataFrame(newRow), ignore_index=True)
            print('空的评论数量： %d'%(nonRatingCount))
            clean_pd.to_csv(ratings_clean_filename)

        # 读取
        if isTrain:
            temp_pd = clean_pd.sample(frac=trainTestRate)
        else:
            temp_pd = clean_pd.sample(frac=trainTestRate)
            temp_pd = clean_pd.drop(temp_pd.index.tolist(), axis=0)

        for i, row in temp_pd.iterrows():
            if max_row != -1 and i > max_row:
                break

            self.label_list.append(torch.tensor(row['labels']).long())
            self.token_list.append(torch.from_numpy(numpy.array( json.loads(row['data']) ) ).long())

    def __getitem__(self, index):
        return self.token_list[index], self.label_list[index]#, self.token_positions

    def __len__(self):
        return len(self.label_list)
    

In [17]:
ratingData = RatingData('../../DataSets/yf_dianping',
                             word2index = word2index,
                            #  max_row= 5,
                             isTrain=True,
                             trainTestRate = 0.8,
                             wordCuter= jieba,
                             clean_file_name='balance_data_4_HAN_means_600000.csv',
                             )
trainLoader = torch.utils.data.DataLoader(dataset=ratingData,
                                          batch_size=128,
                                          shuffle = True,
                                          # num_workers = 8,
                                          )
ratingData2 = RatingData('../../DataSets/yf_dianping',
                             word2index = word2index,
                             # max_row= 200000,
                             isTrain=False,
                             trainTestRate = 0.8,
                             wordCuter= jieba,
                             clean_file_name='balance_data_4_HAN_means_600000.csv',
                             )
testLoader = torch.utils.data.DataLoader(dataset=ratingData2,
                                          batch_size=128,
                                          shuffle = True,
                                          # num_workers = 8,
                                          )
print(len(ratingData.label_list))
print(len(ratingData2.label_list))


 balance_data.csv 所在path: ../../DataSets/yf_dianping
 balance_data.csv 所在path: ../../DataSets/yf_dianping
443310
110828


## 创建模型


In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, num_input, num_hidden):
        super(SelfAttention, self).__init__()
        self.W = nn.Linear(num_input, num_hidden)
        self.U = nn.Linear(num_hidden, 1)

    def forward(self, x):
        u = F.tanh(self.W(x))
        a = F.softmax(self.U(u), dim=1)
        return torch.mul(a, x).sum(dim=1)


class HAN(nn.Module):
    def __init__(self, num_embeddings = 5845,
                 num_classes = 10,
                 num_words = 20,        # 每句话最多多少个词
                 num_sentence = 10,     # 一篇文章多少个句子
                 embedding_dim = 200,
                 hidden_size_gru = 50,
                 hidden_size_att = 100,
                 ):
        super(HAN, self).__init__()

        self.num_words = num_words
        self.num_sentence = num_sentence
        self.embed = nn.Embedding(num_embeddings, embedding_dim, 0)

        self.GRU1 = nn.GRU(embedding_dim,
                           hidden_size_gru,
                           bidirectional=True,  # 双向  Default: ``False``
                           batch_first=True,    # : If ``True``, then the input and output tensors are provided as (batch, seq, feature). Default: ``False``
                           )
        self.self_attention1 = SelfAttention(hidden_size_gru * 2, hidden_size_att)


        self.GRU2 = nn.GRU(hidden_size_gru * 2,
                           hidden_size_gru,
                           bidirectional=True,  # 双向  Default: ``False``
                           batch_first=True,    # : If ``True``, then the input and output tensors are provided as (batch, seq, feature). Default: ``False``
                           )
        self.self_attention2 = SelfAttention(hidden_size_gru * 2, hidden_size_att)

        self.fc = nn.Linear(hidden_size_att, num_classes)

    def forward(self, x:torch.Tensor):
        # print("x:", x.shape, self.num_words)
        sentences = []

        for i in range(self.num_sentence):
            sentence = x[:, i * self.num_words: (i + 1) * self.num_words]
            # print("view 后 x:", x.shape)
            sentence = self.embed(sentence)
            # print("embed 后 x:", x.shape)
            sentence, _ = self.GRU1(sentence)
            # print("GRU1 后 x:", x.shape)
            sentence = self.self_attention1(sentence)
            # print("self_attention1 后 x:", x.shape)
            sentences.append(sentence)
        sentences = torch.cat(sentences)

        x = sentences.view(sentences.size(0) // self.num_sentence, self.num_words, -1)
        # print("view2 后 x:", x.shape)
        x, _ = self.GRU2(x)
        # print("GRU2 后 x:", x.shape)
        x = self.self_attention2(x)
        # print("self_attention2 后 x:", x.shape)
        x = self.fc(x)
        # print("fc 后 x:", x.shape)
        return F.softmax(x, dim=1)


In [28]:

#创建模型
wordEmbedding = torch.FloatTensor(wordEmbedding)
num_embeddings = len(word2vec.wv.index2word)
model = HAN(num_embeddings, 
            num_classes = 3,
            embedding_dim = word2vec.wv.vector_size, 
            num_words = 10,
            num_sentence = 10,
            hidden_size_gru = 200,
            hidden_size_att = 400,
            )
print(model)

modelParams = model.parameters()
for param in modelParams:
    if len(param.data.shape) > 1:
        # print('---', param.data.shape, param.data)
        torch.nn.init.kaiming_normal(param.data)
        # print('--->', param.data)
        
model.embed.from_pretrained(wordEmbedding)


C:\Users\fuliu\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\fuliu\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
C:\Users\fuliu\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:18: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


HAN(
  (embed): Embedding(99999, 200, padding_idx=0)
  (GRU1): GRU(200, 200, batch_first=True, bidirectional=True)
  (self_attention1): SelfAttention(
    (W): Linear(in_features=400, out_features=400, bias=True)
    (U): Linear(in_features=400, out_features=1, bias=True)
  )
  (GRU2): GRU(400, 200, batch_first=True, bidirectional=True)
  (self_attention2): SelfAttention(
    (W): Linear(in_features=400, out_features=400, bias=True)
    (U): Linear(in_features=400, out_features=1, bias=True)
  )
  (fc): Linear(in_features=400, out_features=3, bias=True)
)


Embedding(99999, 200)

## 开始训练过程

In [29]:
def trainOneEpoch(epoch, model:HAN, trainLoader, optimizer:Optimizer, lossFunc):
    if torch.cuda.is_available():
        model = model.cuda()
        lossFunc = lossFunc.cuda()

    model.train()

    startTime = time.time()
    for i, (x, y) in enumerate(trainLoader):
        if torch.cuda.is_available():
            x = x.cuda()
            y = y.cuda()

        outputs = model(x)
        loss = lossFunc(outputs, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 100 == 99:
            print('Epoch %d, %d/%d, loss:%f ' % (epoch, i, len(trainLoader), loss))
        # if i > 2000:
        #     break 
    print('Epoch %d cost time: %.3fs' % (epoch, time.time() - startTime))


def testModel(epoch, model:HAN, testLoader):
    if torch.cuda.is_available():
        model = model.cuda()

    model.eval()

    total = 0
    correct = 0

    startTime = time.time()
    for i, (x, y) in enumerate(testLoader):
        if torch.cuda.is_available():
            x = x.cuda()
            y = y.cuda()

        outputs = model(x)
        _, predicted = torch.max(outputs.data, 1)

        total += len(y)
        correct += predicted.data.eq(y.data).cpu().sum().numpy()
        

        if i % 50 == 49:
            print('Epoch Test %d, %d/%d, \npredicted:%s, \ntarget:%s' % (epoch, i, len(testLoader), 
                                                                     str(predicted),
                                                                     str(y)))
        # if i > 2000:
        #     break 
    print('Epoch Test %d cost time: %.3fs' % (epoch, time.time() - startTime))
    print('准确率： %.3f' % (correct / total))
    return correct / total


def train(epoch, model, modelSavePath, isLoad, lr=0.0002):
    last_acc = 0
    if isLoad: 
        model.load_state_dict(torch.load(modelSavePath))
        last_acc = testModel(epoch, model, testLoader)
    # optimizer=torch.optim.SGD(model.parameters(), momentum=0.9, lr=0.001)
    optimizer=torch.optim.Adam(model.parameters(), lr=0.0002)
    lossFunc =torch.nn.CrossEntropyLoss()
    for epoch in range(epoch):
        trainOneEpoch(epoch, model, trainLoader, optimizer, lossFunc)
        acc = testModel(epoch, model, testLoader)
        if last_acc < acc:
          torch.save(model.state_dict(), modelSavePath)
        last_acc = acc
    

In [30]:

train(10, model, 'EmotionAnalyzeModelData_ClassicalHAN.model', False, lr=0.001)

RuntimeError: CUDA out of memory. Tried to allocate 78.00 MiB (GPU 0; 4.00 GiB total capacity; 2.63 GiB already allocated; 52.75 MiB free; 2.76 GiB reserved in total by PyTorch)

In [0]:

def eval(model, modelSavePath, isLoad = True):
    if isLoad: model.load_state_dict(torch.load(modelSavePath))
    testModel(0, model, testLoader)

eval(model, 'EmotionAnalyzeModelData_300_600.model', isLoad = True)

In [0]:
lossFunc =torch.nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
_, predicted = torch.max(input.data, 1)
output = lossFunc(input, target)
print('input',input, '\n target',target, '\n output', output)
output = lossFunc(input, predicted)
print('predicted',predicted, '\n target',target, '\n output', output)

print(predicted.data.eq(target).cpu().sum())
print(target.data.eq(predicted).cpu().sum())

In [0]:
# 何凯明初始化

w = torch.Tensor(3, 5, 2)
print(w)
print(nn.init.kaiming_uniform(w))
print(w)
w = torch.Tensor(3, 5, 2)
print(w)
print(torch.nn.init.kaiming_normal(w))
print(w)